In [68]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os
import joblib

def find_and_load_models(root_folder):
    models_scalers_list = [] # Liste pour stocker les modèles et scalers chargés
    for root, dirs, files in os.walk(root_folder):
        for dir in dirs:
            if dir.startswith("best_model_"):
                model_path = os.path.join(root, dir, "model.pkl")
                # Le scaler est au même niveau que le dossier du modèle, pas à l'intérieur
                scaler_path = os.path.join(root, "scaler.joblib")  # Chemin ajusté pour le scaler
                if os.path.exists(model_path):
                    model = joblib.load(model_path)  # Chargement du modèle
                    model_name = dir.replace("best_model_", "")
                    model_info = {
                        "model_name": model_name,
                        "model": model,  # Le modèle chargé
                        "model_path": model_path,  
                        "root": root
                    }

                    if os.path.exists(scaler_path):
                        scaler = joblib.load(scaler_path)
                        model_info["scaler"] = scaler
                    else:
                        model_info["scaler"] = None
                    models_scalers_list.append(model_info)
    return models_scalers_list

root_folder = "0" 
models_scalers_list = find_and_load_models(root_folder)

# Pour lister les modèles et scalers chargés
for model_info in models_scalers_list:
    model_name = model_info["model_name"]
    scaler_status = 'Présent' if model_info["scaler"] is not None else 'Absent'
    print(f"Modèle: {model_name}, Scaler: {scaler_status}, Chemin: {model_info['model_path']}")


Modèle: lgbm, Scaler: Présent, Chemin: 0\04ba12c54d91485885c7f556699a7222\artifacts\best_model_lgbm\model.pkl
Modèle: lgbm, Scaler: Présent, Chemin: 0\11f95b5491ca42119377ca950bbfcf19\artifacts\best_model_lgbm\model.pkl
Modèle: rf, Scaler: Présent, Chemin: 0\17e8cef71d8149398a80f1ef3446e500\artifacts\best_model_rf\model.pkl
Modèle: lgbm, Scaler: Présent, Chemin: 0\505247fea4f540f791c93b24c24a2453\artifacts\best_model_lgbm\model.pkl
Modèle: rf, Scaler: Présent, Chemin: 0\73c18e6cb2ab4afda84bbee1e6868e82\artifacts\best_model_rf\model.pkl
Modèle: rf, Scaler: Présent, Chemin: 0\8821b19bad7b48158c4507cc21bee8de\artifacts\best_model_rf\model.pkl
Modèle: lgbm, Scaler: Présent, Chemin: 0\959da293c7df4267b09aff5e4f41d308\artifacts\best_model_lgbm\model.pkl
Modèle: rf, Scaler: Présent, Chemin: 0\a8e06511aa974e7bb8040be54b15c65c\artifacts\best_model_rf\model.pkl
Modèle: gbc, Scaler: Présent, Chemin: 0\af6cde65cb4c484195b98a0c3fd532cd\artifacts\best_model_gbc\model.pkl
Modèle: lgbm, Scaler: Présen

In [69]:
data_path = "../../2024_data/Machine_14/data/dataset_for_training_criticality_balanced_08.1.csv"
df = pd.read_csv(data_path)
df.drop(columns=["Unnamed: 0", "leftMargin_remoteScannerRegistration","optifoil_ifoil"], inplace=True)
df.rename(columns={"varnishConsumptionVarnish_3d": "varnishConsumptionVarnish_3d_job"}, inplace=True)
pd.set_option('display.max_columns', None)
df.head()

,duration,speed,paperHeight_job,paperWidth_job,total_copies_requested,ifoil,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,LED,drops,speed_ifoil,heater1Enabled_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,criticality_events
0,74.200,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,ERROR
1,74.200,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,ERROR
2,308.975,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,INFO
3,308.975,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,INFO
4,197.449,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,INFO


In [70]:
df.drop_duplicates(inplace=True)

In [71]:
df.head()

,duration,speed,paperHeight_job,paperWidth_job,total_copies_requested,ifoil,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,LED,drops,speed_ifoil,heater1Enabled_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,criticality_events
0,74.200,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,ERROR
2,308.975,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,INFO
4,197.449,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,INFO
5,197.449,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,WARNING
8,128.315,500,520,740,350,False,1,2,0.0,10,1,36.0,False,115,70,0,45,100,2561,2561,2561,16,16,WARNING


In [72]:
# Créer une nouvelle colonne 'binary_class' avec des valeurs booléennes
df['error'] = df['criticality_events'].apply(lambda x: True if x == 'ERROR' else False)
   
print(df['criticality_events'].value_counts())


df.drop(columns=['criticality_events'], inplace=True)


X_test = df.drop(columns=["error"], axis=1)
y_test = df["error"]



print("Distribution des classes dans y_test :")
print(y_test.value_counts())

INFO        118
WARNING     105
NO_EVENT    103
ERROR        95
Name: criticality_events, dtype: int64
Distribution des classes dans y_test :
False    326
True      95
Name: error, dtype: int64


In [73]:
def evaluate_models(models_scalers, X_test, y_test):
    results = []
    highest_accuracy = 0
    best_model_name = ""
    best_model_predictions = []
    best_model_precision = 0
    best_model_recall = 0
    
    for model_info in models_scalers:
        model_name = model_info["model_name"]
        model = model_info["model"] 
        scaler = model_info.get("scaler")  
        
        # print(f"\nÉvaluation du modèle: {model_name}")
        
        # Identifier les caractéristiques numériques dans X_test
        numeric_features = X_test.select_dtypes(include=np.number).columns
        X_test_scaled = X_test.copy()
                
        # Appliquer le scaler uniquement aux caractéristiques numériques si un scaler est présent
        if scaler is not None:
            # print(f"Application du scaler pour le modèle {model_name}.")
            X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])
            # print("Aperçu des données de test après mise à l'échelle pour les premières lignes :")
            # print(X_test_scaled[numeric_features].head())

        # Prédire et calculer l'accuracy
        y_pred = model.predict(X_test_scaled)
        # print(f"Prédictions pour le modèle {model_name} sur les 10 premiers échantillons :", y_pred[:10])
        accuracy = accuracy_score(y_test, y_pred)
        # print(f"Accuracy pour {model_name}: {accuracy}\n")
        
        # Calculer la précision et le recall pour la classe True
        precision = classification_report(y_test, y_pred, output_dict=True)["True"]["precision"]
        recall = classification_report(y_test, y_pred, output_dict=True)["True"]["recall"]
        
        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            best_model_name = model_name
            best_model_predictions = y_pred
            best_model_precision = precision
            best_model_recall = recall
        
        results.append({"Model Name": model_name, "Accuracy": accuracy, "Precision (True)": precision, "Recall (True)": recall})
    
    # Créer un DataFrame pour les résultats et les trier par accuracy
    results_df = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)
    print("Résultats finaux de l'évaluation des modèles :")
    print(results_df)
    
    # Classification report 
    print(f"\nClassification report pour le modèle avec la plus haute accuracy ({best_model_name}):")
    print(classification_report(y_test, best_model_predictions))
    
    print(f"\nModèle avec la plus haute précision pour la classe True: {best_model_name}")
    print(f"Précision pour la classe True: {best_model_precision}")
    
    print(f"\nModèle avec le recall le plus élevé pour la classe True: {best_model_name}")
    print(f"Recall pour la classe True: {best_model_recall}")
    
    return results_df

evaluate_models(models_scalers_list, X_test, y_test)


Résultats finaux de l'évaluation des modèles :
   Model Name  Accuracy  Precision (True)  Recall (True)
0         gbc  0.750594          0.222222       0.042105
1          rf  0.738717          0.285714       0.105263
2        lgbm  0.669834          0.255556       0.242105
3        lgbm  0.669834          0.255556       0.242105
4        lgbm  0.669834          0.255556       0.242105
5        lgbm  0.667458          0.281553       0.305263
6        lgbm  0.667458          0.258065       0.252632
7        lgbm  0.667458          0.258065       0.252632
8        lgbm  0.667458          0.258065       0.252632
9         xgb  0.650831          0.279661       0.347368
10       lgbm  0.650831          0.234694       0.242105
11         rf  0.577197          0.254438       0.452632
12         rf  0.543943          0.225989       0.421053
13         rf  0.541568          0.239362       0.473684
14         rf  0.541568          0.239362       0.473684

Classification report pour le modèle ave

,Model Name,Accuracy,Precision (True),Recall (True)
0,gbc,0.750594,0.222222,0.042105
1,rf,0.738717,0.285714,0.105263
2,lgbm,0.669834,0.255556,0.242105
3,lgbm,0.669834,0.255556,0.242105
4,lgbm,0.669834,0.255556,0.242105
5,lgbm,0.667458,0.281553,0.305263
6,lgbm,0.667458,0.258065,0.252632
7,lgbm,0.667458,0.258065,0.252632
8,lgbm,0.667458,0.258065,0.252632
9,xgb,0.650831,0.279661,0.347368


In [74]:
# Extraction du modèle et du scaler pour le modèle 'rf' (assumé à l'index 1)
rf_model_info = models_scalers_list[1]  # Assumant que l'index 1 correspond au modèle 'rf'
rf_model = rf_model_info['model']
rf_scaler = rf_model_info.get('scaler', None)

# Préparation des données de test (X_test_scaled) si un scaler est utilisé
X_test_scaled = X_test.copy()
if rf_scaler is not None:
    numeric_features = X_test.select_dtypes(include=np.number).columns
    X_test_scaled[numeric_features] = rf_scaler.transform(X_test[numeric_features])

# Prédiction à l'aide du modèle 'rf'
y_pred_rf = rf_model.predict(X_test_scaled)

# Ajout des prédictions au DataFrame original
df['prediction'] = y_pred_rf  

# Exportation du DataFrame en CSV avec les prédictions
df.to_csv("predict_vs_real.csv", index=False)

In [75]:
df[['error', 'prediction']]


,error,prediction
0,True,True
2,False,True
4,False,True
5,False,True
8,False,True
...,...,...
531,False,False
532,True,False
533,False,False
534,True,False


In [76]:
# Calcul des True Positives (TP) et False Positives (FP)
TP = ((df['error'] == True) & (df['prediction'] == True)).sum()
FP = ((df['error'] == False) & (df['prediction'] == True)).sum()
FN = ((df['error'] == True) & (df['prediction'] == False)).sum()


# Calcul du nombre total de prédictions True et le nombre total de vrais cas True
total_true_predictions = (df['prediction'] == True).sum()
total_actual_trues = (df['error'] == True).sum()

# Calcul de l'accuracy pour les prédictions True
true_prediction_accuracy = TP / total_true_predictions if total_true_predictions else 0

# Affichage des résultats avec les nouvelles informations
print(f"Nombre total de cas 'True' réels (erreurs qui sont vraiment arrivées dans ce csv) : {total_actual_trues}")
print(f"Nombre total de prédictions 'True'(nombre de fois où le modèle a prédit une erreur sur ce csv) : {total_true_predictions}")
print(f"Parmi ces prédictions 'True', le modèle a eu raison {TP} fois. (nombre de fois où il y a vraiment eu une erreur et le modèle l'a prédit correctement, vrai positif)")
print(f"Le modèle a manqué {FN} erreurs. (nombre de fois où une erreur est survenue mais le modèle ne l'a pas prédit, faux négatif)")

if FP > 0:
    print(f"Le modèle a également fait {FP} fausses prédictions 'True' (fausses alertes, a prédit une erreur alors qu'il n'y en avait pas, faux positif).")

print(f"Le modèle a donc correctement identifié {TP} erreurs sur {total_actual_trues} erreurs totales et loupé {FN} erreurs sur {total_actual_trues} erreurs totales.")

print(f"Pour résumé , l'accuracy des prédictions 'True' est de : {true_prediction_accuracy:.2f} (ou {true_prediction_accuracy*100:.2f}%), le modèle correctement prédit un peu plus de 1 erreur sur 4")


Nombre total de cas 'True' réels (erreurs qui sont vraiment arrivées dans ce csv) : 95
Nombre total de prédictions 'True'(nombre de fois où le modèle a prédit une erreur sur ce csv) : 93
Parmi ces prédictions 'True', le modèle a eu raison 24 fois. (nombre de fois où il y a vraiment eu une erreur et le modèle l'a prédit correctement, vrai positif)
Le modèle a manqué 71 erreurs. (nombre de fois où une erreur est survenue mais le modèle ne l'a pas prédit, faux négatif)
Le modèle a également fait 69 fausses prédictions 'True' (fausses alertes, a prédit une erreur alors qu'il n'y en avait pas, faux positif).
Le modèle a donc correctement identifié 24 erreurs sur 95 erreurs totales et loupé 71 erreurs sur 95 erreurs totales.
Pour résumé , l'accuracy des prédictions 'True' est de : 0.26 (ou 25.81%), le modèle correctement prédit un peu plus de 1 erreur sur 4
